In [1]:
#IPython is what you are using now to run the notebook
import IPython
print( "IPython version:      %6.6s (need at least 1.0)" % IPython.__version__)

# Numpy is a library for working with arrays and matrices
import numpy as np
print( "Numpy version:        %6.6s (need at least 1.7.1)" % np.__version__)

# SciPy implements many different numerical algorithms
import scipy as sp
print( "SciPy version:        %6.6s (need at least 0.12.0)" % sp.__version__)

# Pandas makes working with data tables easier
import pandas as pd
print( "Pandas version:       %6.6s (need at least 0.11.0)" % pd.__version__)

# Module for plotting
import matplotlib.pyplot as plt  
from pylab import *
print( "Matpltolib version:    %6.6s (need at least 1.2.1)" %
       matplotlib.__version__)
%matplotlib inline
# necessary for in-line graphics

# SciKit Learn implements several Machine Learning algorithms
import sklearn
print("Scikit-Learn version: %6.6s (need at least 0.13.1)" %
       sklearn.__version__)
import os
# for certain system-related functions

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso

IPython version:       7.8.0 (need at least 1.0)
Numpy version:        1.16.5 (need at least 1.7.1)
SciPy version:         1.3.1 (need at least 0.12.0)
Pandas version:       0.25.1 (need at least 0.11.0)
Matpltolib version:     3.1.1 (need at least 1.2.1)
Scikit-Learn version: 0.21.3 (need at least 0.13.1)


## World Values Survey

World Value Survey (WVS) is a large survey, conducted in many countries simultaneously. It revolves
around public opinion about traditions, economy, politics, life and other things. I recommend you to
consult the official questionnaire (uploaded in canvas/files/wvs).
In this task the central question is V23: "All things considered, how satisfied are you with your life as a
whole these days?" with answers ranging between 1 (completely dissatisfied) and 10 (completely satisfied).

We are going to model this variables using linear regression.

### 1 Explore and clean the data
First, let's load data and take a closer look at it.

1. Browse the WVS documentation and make sure you are familiar with coding of the variable V23.
Note: you also have to consult the codebook to understand all the missings and how to remove those.

In [2]:
# Reading the world Value data
wvs_df = pd.read_csv('C:/Users/vaide/Documents/574-DS2/ps4/wvs.csv.bz2', sep ='\t')

#Exploring the data
print("\nThe number of rows are", wvs_df.shape[0],"and the number of columns are",wvs_df.shape[1])


The number of rows are 90350 and the number of columns are 328


2. Load the data. Remove all the missing observations of V23. I mean all the missings, including the
valid numeric codes that denote missing/invalid answers.

In [3]:
# Checking the unique values of V23
print(wvs_df.V23.unique())

[ 8  5  4  7  6  3 10  1  9 -1 -2  2 -5]


In [4]:
#Removing missing and invalid values by removing negative values and values greater than 10
wvs_df = wvs_df[(wvs_df.V23 > 0) & (wvs_df.V23 < 11)]
print(wvs_df.V23.unique())

[ 8  5  4  7  6  3 10  1  9  2]


3. Now make a table (or a plot) of different answers. What is the mean satisfaction level on this planet? How large a proportion of people are at 6 or more satisfied?

(Note: without knowing more about how the sample was created, we should not talk about the
planet. We should refer to "respondents" instead.)

In [5]:
#Counting the response for each rating
sat_life_df = pd.DataFrame(wvs_df.groupby('V23').V23.count())
#Renaming the column
sat_life_df.columns = ["Count"]
#Creating the proportion percentage column
sat_life_df["Proportion_percentage"] = sat_life_df["Count"] *100 / len(wvs_df)
sat_more_than_5_df=sat_life_df[(sat_life_df.index>5)].sum()
print("The mean satisfaction level of the respondents is", round(wvs_df.V23.mean(),2))
print("\nThe proportion of respondents satisfied at a 6 or higher level is",round(sat_more_than_5_df.Proportion_percentage,2),"%")
sat_life_df

The mean satisfaction level of the respondents is 6.83

The proportion of respondents satisfied at a 6 or higher level is 73.03 %


,Count,Proportion_percentage
V23,,
1,2828,3.150238
2,1985,2.211182
3,3463,3.857593
4,4600,5.124149
5,11331,12.622116
6,10666,11.881343
7,15493,17.258357
8,18213,20.288289
9,9264,10.319591


## 2 Create the Design Matrix

Now it is time to make the data suitable for a regression model. So far we have either used R-style formulas, or fed data into a ML model directly without much preparatory work. Now it is time to construct the design matrix manually. In case of linear regression, the design matrix is the data matrix that will be directly fed into the formula (X|^T.X)-1X|y, or any function that uses this or another similar formula. Design matrix can also be fed directly into other kind of models, such as logistic regression or decision tree. Design matrix is also needed by various libraries, in particular sklearn's LinearRegression, Ridge, and Lasso. There is an example of how to create a design matrix in the lecture notes, Section 2.1.7.

Your task is to add the variables to the design matrix, one-by-one, and each time doing the necessary encoding if appropriate.

 Many variables are categorical. For instance, variable V2, country, is numeric with different numbers representing different countries. So in essence it is a categorical variable where categories are coded as numbers. The same is true for V80, most serious problem in the world. You should convert such variables to dummies (do your still remember pd.get_dummies?) and remove the original variable. But don't forget to remove missings!

 A large number of variables contain ordered values instead. For instance, V55 asks how much choice do you feel do you have over your life. The answers range from 1 (no choice at all) to 10 (a great deal of choice). We treat these as numeric response. Although, strictly speaking not correct, the model would be too messy if we were creating a category for each response. However, the missings (-5: inapplicable, -4: not asked etc) are not ordered in any meaningful sense. Hence your task is to
remove missings.

Note that many variables, e.g. v74b (important to help people nearby) and v90 (signing petition), contain a very large number of missings, and hence you essentially lose all your data if you include such variables. So you should remove such variables and replace with others that have more valid answers.

Was it clear? Good. Enough of talk, let's dive into the real thing.

1. Create your outcome variable y out of life satisfaction V23 (remove missings!)
2. Create a design matrix that contains at least 100 variables from the WVS data. Your selected variables should contain at least a few categorical ones, such as V2 country. In each case:
(a) remove missing observations
(b) convert categorical variable to dummies if appropriate. Don't forget to drop the reference
category.

This will result in a large amount of code that is essentially copy-paste, but not exactly. It is a little
bit tedious to do though. Think about options to make it more automatic but... hint: there are no
good options...
Note that when converting 100 variables into a design matrix, the latter may end up with many
more columns.
Keep in mind that you end up deleting quite a few observations, everything that contains missings in any
of your selected variables.

Response: 

The below code orders columns in relation to the number of the missing/negative values they have.

In [6]:
column_pos_count_dict = {}
for i in range(wvs_df.shape[1]):
    column_pos_count_dict[wvs_df.iloc[:,i].name] = len(wvs_df[wvs_df.iloc[:,i]>0])

In [7]:
# Sorting according to the number of positive responses
sorted_pos_count_dict = dict(sorted(column_pos_count_dict.items(), key=lambda item: item[1], reverse = True))
sorted_pos_count_dict

{'V2': 89771,
 'V23': 89771,
 'V22': 89769,
 'V13': 89766,
 'V14': 89765,
 'V16': 89763,
 'V18': 89761,
 'V17': 89760,
 'V12': 89759,
 'V44': 89759,
 'V20': 89758,
 'V15': 89756,
 'V19': 89754,
 'V21': 89754,
 'V240': 89689,
 'V242': 89602,
 'V57': 89564,
 'V11': 89503,
 'V4': 89482,
 'V5': 89300,
 'V59': 89254,
 'V10': 89124,
 'V248': 88954,
 'V84': 88878,
 'V6': 88837,
 'V80': 88755,
 'V225': 88626,
 'V200': 88557,
 'V55': 88542,
 'V9': 88523,
 'V188': 88516,
 'V82': 88507,
 'V209': 88478,
 'V208': 88472,
 'V210': 88454,
 'V102': 88443,
 'V179': 88418,
 'V211': 88377,
 'V8': 88336,
 'V45': 88286,
 'V7': 88259,
 'V143': 88240,
 'V37': 88234,
 'V39': 88234,
 'V229': 88229,
 'V202': 88223,
 'V199': 88207,
 'V191': 88198,
 'V190': 88190,
 'V189': 88047,
 'V177': 88008,
 'V250': 87994,
 'V104': 87973,
 'V170': 87955,
 'V111': 87953,
 'V113': 87877,
 'V103': 87861,
 'V79': 87850,
 'V72': 87788,
 'V140': 87786,
 'V176': 87775,
 'V100': 87705,
 'V98': 87704,
 'V78': 87645,
 'V77': 87631,
 'V

The below code removes all missing observations from the selected columns including V23.

In [8]:
# selecting 105 varibles for design matrix
top_var_list=list(sorted_pos_count_dict.keys())[:105] 
var_df=wvs_df[top_var_list]
for i in range(var_df.shape[1]):
     var_df = var_df[var_df.iloc[:,i] > 0]

Converting the categorical variables V2 and V80 into dummies and dropping one dummy column to avoid the problem of perfect multicollinearity

In [9]:
var_df.rename(columns={'V2':'country'}, inplace=True)
var_df=pd.get_dummies(var_df, columns = ['country'])
var_df = var_df.drop("country_887", axis = 1)

In [10]:
var_df.rename(columns={'V80':'problem'}, inplace=True)
var_df=pd.get_dummies(var_df, columns = ['problem'])
var_df = var_df.drop("problem_5", axis = 1)

2. Create your outcome variable y out of life satisfaction V23 (remove missings!)

In [11]:
y_res = var_df.V23  #This column is clean as we have already removed missing and invalid values
design_matrix_df=var_df.drop("V23",axis=1)

### 3 Condition numbers

1. Compute the condition number of your output matrix in such a manner.

In [12]:
columns = design_matrix_df.columns
array_new = list(columns[:1])
columns = columns[1:]
col_cond_num = {}
for i in columns:
    array_new.append(i)
    col_cond_num[i] = np.linalg.cond(np.array(var_df[array_new]))

In [13]:
print("Our design matrix has",design_matrix_df.shape[0],"rows and",design_matrix_df.shape[1],"columns.")

Our design matrix has 48888 rows and 158 columns.


In [14]:
for each_col in col_cond_num:
    print("The condition number after adding the" , each_col , "column is", col_cond_num[each_col])

The condition number after adding the V13 column is 4.59448341840832
The condition number after adding the V14 column is 5.75185702638533
The condition number after adding the V16 column is 6.584943179033798
The condition number after adding the V18 column is 7.520200660827082
The condition number after adding the V17 column is 8.358192772203937
The condition number after adding the V12 column is 9.057570842826395
The condition number after adding the V44 column is 11.354582828061679
The condition number after adding the V20 column is 12.263481406976856
The condition number after adding the V15 column is 13.195221854153317
The condition number after adding the V19 column is 14.095042580999253
The condition number after adding the V21 column is 14.754836638260839
The condition number after adding the V240 column is 15.308114493523748
The condition number after adding the V242 column is 118.3202382211488
The condition number after adding the V57 column is 118.47377821186518
The condition

Our final design matrix has a decent condition number approximately equal to 4596 and is well-suitable for analysis.

## 4 Do Some Social Science

Before getting further, let's do a simple social science analysis. How is life satisfaction related to health (v11), perceived control over life (v55 ) and financial situation (v59 )? Let's analyze association between satisfaction and just these three variables.

1. run a linear regression models explaining satisfaction with these three variables. Present the output table. I recommend to use statsmodels.formula.api for this task (but you have to use sklearn later).

In [15]:
import statsmodels.formula.api as smf

soc_science_df = wvs_df[['V23','V11','V55','V59']]
soc_science_df = soc_science_df[soc_science_df.V11 > 0]
soc_science_df = soc_science_df[soc_science_df.V55 > 0]
soc_science_df = soc_science_df[soc_science_df.V59 > 0]

train_data = soc_science_df[:70391]
test_data = soc_science_df[70391:]
  
regr = smf.ols(formula='V23~V11+V55+V59',data=train_data)
fitmod=regr.fit() #fitting the model

2. comment the output table in terms of relative effect size and statistical significance. Any surprises for you?

Response:

From the below model summary we can see that all 3 response variables are statistically significant as the p-values are lesser than 0.05 and also the model has a large F-statistic value which means that the model is statistically significant. The model shows that when the variable V11 decreases in value (i.e. the state of health is very good), the life statisfaction increases. Similarly, when the financial situation of the respondents and their peceived control over life increases, the life satisfaction increases. This model also has a condition number of 50 which means that our training data is well-suited for modelling.

However, the R-squared value for the model is 0.32 which is a bit low and it implies that the model cannot explain the variance in the dataset.

In [16]:
fitmod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    V23   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.320
Method:                 Least Squares   F-statistic:                 1.104e+04
Date:                Fri, 06 Mar 2020   Prob (F-statistic):               0.00
Time:                        16:17:14   Log-Likelihood:            -1.4427e+05
No. Observations:               70391   AIC:                         2.886e+05
Df Residuals:                   70387   BIC:                         2.886e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.7572      0.036    104.743      0.000       3.687       3.827
V11           -0.3958      0.009    -46.125      0.000      -0.413      -0.379
V55            0.2469      0.003     73.055      0.000       0.240       0.253
V59            0.3573      0.003    115.334      0.000       0.351       0.363
==============================================================================
Omnibus:                     3306.564   Durbin-Watson:                   1.687
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5618.192
Skew:                          -0.392   Prob(JB):                         0.00
Kurtosis:                       4.141   Cond. No.                         50.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3. compute and present RMSE (just on training data). This will serve as the benchmark for the future.

In [17]:
print("The RMSE on the training dataset is",sqrt(fitmod.mse_resid))

The RMSE on the training dataset is 1.8788670435909323


## 5 Back to ML: Model

Now it is time to use all these variables to model satisfaction. Use sklearn.linear_model.LinearRegression here as this is easy to be switched with ridge and lasso, and it takes in the design matrix directly.


1. compute the condition number for your design matrix (just a single number, not the stepwise procedure).

In [18]:
print("Condition number of X is :", np.linalg.cond(design_matrix_df))

Condition number of X is : 4595.663117943389


2. Split the data into training-validation chunks (80-20 or so)

In [19]:
# Splitting the data into training and testing data 
X_train, X_test, y_train, y_test = train_test_split(design_matrix_df, y_res, test_size = 0.2)

3. compute the condition number for your training design matrix (just a single number, not the stepwise procedure).

In [20]:
print("Condition number of X-train is :", np.linalg.cond(X_train))

Condition number of X-train is : 4630.745564967672


4. Fit a linear regression model where you describe satisfaction with the design matrix X you just created.

In [21]:
regr = LinearRegression().fit(X_train, y_train) 

5. predict and compute RMSE on training data

In [22]:
yhat_train = regr.predict(X_train)
rmse_train = np.sqrt(np.mean((y_train - yhat_train)**2))
print("RMSE for linear regression for training data is :", rmse_train)

RMSE for linear regression for training data is : 1.6787746754375492


6. predict and compute RMSE on testing data

In [23]:
yhat_test = regr.predict(X_test)
rmse_test = np.sqrt(np.mean((y_test - yhat_test)**2))
print("RMSE for linear regression for testing data is :", rmse_test)

RMSE for linear regression for testing data is : 1.694504997938269


7. Repeat the previous with Ridge regression, play a little with different alpha-s. Which if gave you the best testing RMSE? (No need for a rigorous analysis, just play a little)

In [24]:
#large alpha = large penalty
alphas= [0.1,0.5,1,3, 10,100]

for alpha in alphas:
    mr = Ridge(alpha=alpha).fit(X_train, y_train)
    yhatr = mr.predict(X_test)
    rmser = np.sqrt(np.mean((y_test - yhatr)**2))
    print("RMSE for Ridge for alpha" , alpha ,"is :", rmser)

RMSE for Ridge for alpha 0.1 is : 1.6944995638586497
RMSE for Ridge for alpha 0.5 is : 1.6944812870064208
RMSE for Ridge for alpha 1 is : 1.6944642122118978
RMSE for Ridge for alpha 3 is : 1.6944269656112967
RMSE for Ridge for alpha 10 is : 1.6943819455789368
RMSE for Ridge for alpha 100 is : 1.6949582826433416


Thus, we can see that a alpha of 3 gave us the lowest RMSE for the model using Ridge regression.

8. And repeat with Lasso regression again playing a little with different alpha-s.

In [25]:
alphas = [0.1,0.5,1,3,10,100]
for alpha in alphas:
    ml = Lasso(alpha=alpha).fit(X_train, y_train)
    yhatl = ml.predict(X_test)
    rmsel = np.sqrt(np.mean((y_test - yhatl)**2))
    print("RMSE for Lasso for alpha", alpha, "is :", rmsel)

RMSE for Lasso for alpha 0.1 is : 1.7521929138180097
RMSE for Lasso for alpha 0.5 is : 1.9037143888620092
RMSE for Lasso for alpha 1 is : 1.9670724526062617
RMSE for Lasso for alpha 3 is : 2.2354242613837343
RMSE for Lasso for alpha 10 is : 2.2354242613837343
RMSE for Lasso for alpha 100 is : 2.2354242613837343


Thus, we can see that the smallest alpha value of 0.1 gave us the lowest RMSE for the model using Lasso regression.

9. comment your results:


(a) Compare RMSE on testing/training data. What does this suggest in terms of overfitting?

Answer: The model has a similar RMSE on the training and test data. These means that our model has a good fit and can predict correct values on unseen data as well and has not overfit our training data.

(b) Compare RMSE for OLS, Ridge and Lasso

Answer: The RMSE for OLS, Ridge and Lasso are almost similar in value. This shows that regularization did not add any value to as our initial model did not overfit the data.

(c) Compare the resulting RMSE with the small benchmark model you did above

Answer: By comparing the RMSE of the small benchmark model and of models above, we can see that there is a very minute difference between the RMSE's of the model with 3 predictors and the model with over 150 predictors. Thus we can say that the extra 150 variables added very little explanatory power to the model and were not relevant for analysis.

## 6 Let's Overfit!

As WVS is a relatively large dataset we cannot easily overfit by adding more variables. But we can go another easy route instead: we take a subsample.

1. Create a subsample of your design matrix and the outcome variable. Choose a large-ish sample that overfits. The size depends on which variables do you exactly choose, in my case 2000 obs rarely overfits (it depends on the train-validation split), 1000 typically overfits.
2. repeat the steps you did above.
3. comment how do OLS, Ridge, Lasso perform on testing/training in case of overfitting.
4. comment the condition number of design matrix and overfitting.


In [26]:
wvs_sample_df = pd.concat([design_matrix_df, y_res] , axis = 1)
wvs_sample_df = wvs_sample_df.sample(n = 1000)

2. repeat the steps you did above.

In [27]:
def ml_model(X , y):
    print("Condition number of X is :", np.linalg.cond(X))
    # Splitting the data into training and testing data 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)  
    print("Condition number of X-train is :", np.linalg.cond(X_train))
    regr = LinearRegression().fit(X_train, y_train) 
    #print("sklearn estimated coefficients:\n", regr.coef_)
    yhat_train = regr.predict(X_train)
    yhat_test = regr.predict(X_test)
    rmse_train = np.sqrt(np.mean((y_train - yhat_train)**2))
    rmse_test = np.sqrt(np.mean((y_test - yhat_test)**2))
    print("\nRMSE for linear regression for training data is :", rmse_train)
    print("RMSE for linear regression for testing data is :", rmse_test)
    
    alphas= [0.1,0.5,1,3,10,100]  # sometimes refer to this as lambda
    for alpha in alphas:
        mr = Ridge(alpha=alpha).fit(X_train, y_train)
        yhatr_train = mr.predict(X_train)
        yhatr_test = mr.predict(X_test)
        rmser_train = np.sqrt(np.mean((y_train - yhatr_train)**2))
        rmser_test = np.sqrt(np.mean((y_test - yhatr_test)**2))
        print("\nRMSE for Ridge for alpha ", alpha, " for training data is :", rmser_train)
        print("RMSE for Ridge for alpha ", alpha, " for testing data is :", rmser_test)  
   
    for alpha in alphas:
        ml = Lasso(alpha=alpha).fit(X_train, y_train)
        yhatl_train = ml.predict(X_train)
        yhatl_test = ml.predict(X_test)
        rmsel_train = np.sqrt(np.mean((y_train - yhatl_train)**2))
        rmsel_test = np.sqrt(np.mean((y_test - yhatl_test)**2))
        print("\nRMSE for Lasso for alpha ", alpha, " for training data is :", rmsel_train)
        print("RMSE for Lasso for alpha ", alpha, " for testing data is :", rmsel_test)

In [28]:
ml_model(wvs_sample_df.loc[:, wvs_sample_df.columns != 'V23'], wvs_sample_df.V23)

Condition number of X is : 5219.945188462191
Condition number of X-train is : 4944.429146593619

RMSE for linear regression for training data is : 1.4373454541904003
RMSE for linear regression for testing data is : 1.9493014818979681

RMSE for Ridge for alpha  0.1  for training data is : 1.438122347250804
RMSE for Ridge for alpha  0.1  for testing data is : 1.954106902236144

RMSE for Ridge for alpha  0.5  for training data is : 1.4396086704188784
RMSE for Ridge for alpha  0.5  for testing data is : 1.9552118772770417

RMSE for Ridge for alpha  1  for training data is : 1.4407819854559494
RMSE for Ridge for alpha  1  for testing data is : 1.953658888701536

RMSE for Ridge for alpha  3  for training data is : 1.4465409559530011
RMSE for Ridge for alpha  3  for testing data is : 1.9483956059535357

RMSE for Ridge for alpha  10  for training data is : 1.4663478024140209
RMSE for Ridge for alpha  10  for testing data is : 1.9397651077789044

RMSE for Ridge for alpha  100  for training data

3. Comment how do OLS, Ridge, Lasso perform on testing/training in case of overfitting.

Answer:

On overfitting the data we have the below observations for all the models:

1) OLS: The RMSE on the training data is lower than that of the testing data. This means that our model is overfitting our trainng data as it cannot predict correct values on unseen data.

2) Ridge: For alpha of 100, we can see that the RMSE of the test data is the lowest but it is still a little higher than that of the training data. However, this is lower than that of the OLS model. This shows that using regularization we can reduce the overfitting of the model caused by simple linear regression.

3) Lasso: For alpha of 0.1, we can see that the RMSE of the test data is the lowest but it is still higher than that of the training data. However, this is lower than that of the OLS model. This shows that using regularization we can reduce the overfitting of the model caused by simple linear regression.